***(i) Import the function from the module "mzprojection"***

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
# from mzprojection import mzprojection_multivariate, calc_correlation
from mzprojection import mzprojection_multivariate_long_time_series, calc_correlation_long_time_series, calc_residual_long_time_series

#help(mzprojection_multivariate_long_time_series)

***(ii) Prepare a long-time-series data***  
Here, instead of an ensemble set of short-time-series data, we use a long time-series data by assuming ergodicity.

In [ ]:
#= Read sample data =
indata = np.loadtxt('../sample_data/sample_time_series.dat')
t_raw     = indata[:,0]                     # Time t
u_raw    = indata[:,1] + 1.0j * indata[:,2] # Variable of interest u(t)
dudt_raw = indata[:,3] + 1.0j * indata[:,4] # = du/dt
f_raw    = indata[:,5] + 1.0j * indata[:,6] # Analyzed data f(t)

fig = plt.figure(figsize=(14,2.5))
ax = fig.add_subplot(111)
ax.set_xlim(0,1000)
ax.set_xticks(np.arange(0,1001,100))
ax.set_ylim(-0.1,0.1)
ax.set_xlabel("Time t")
ax.set_ylabel("Long time-series u(t), f(t)")
ax.plot(t_raw,f_raw.real*10,label="Re[f(t)]x10")
ax.plot(t_raw,u_raw.real,label="Re[u(t)]")
plt.legend()
plt.show()

***(iii) Apply the Moti-Zwanzig projection operator method***  
The projection of $f(t)^i$ on $u(t)^i$ is calculated as,  
    $f(t)=\Omega u(t)+s(t)+r(t)$,  
    $s(t)=-\int_0^t \Gamma(t) u(t-v)dv$.  
The Markov coefficient $\Omega$, the memory function $\Gamma(t)$ and the uncorrelated term $r(t)$ are obtained as outputs.  
(Some correlations, e.g., $\langle r(t) u \rangle$ are also obtained to check the result.)

In [ ]:
delta_t = t_raw[1] - t_raw[0] # Time step size
ista=2000    # Skip time steps in the beginning
t=t_raw[ista:]
u=u_raw[ista:]
dudt=dudt_raw[ista:]
f=f_raw[ista:]
nperiod=500  # Length of time steps to evaluate the memory function
omega, memoryf, s, r = mzprojection_multivariate_long_time_series(delta_t, u, dudt, f, nperiod=nperiod, flag_terms=True, flag_debug=True)
t_cor = delta_t * np.arange(memoryf.shape[0])

### Reshape for single variable
omega = omega.reshape(1)
memoryf = memoryf.reshape(nperiod)
              
uu = calc_correlation_long_time_series(u,u,nperiod)[:,0,0]
ududt = calc_correlation_long_time_series(u,dudt,nperiod)[:,0,0]
ff = calc_correlation_long_time_series(f,f,nperiod)[:,0,0]
fdudt = calc_correlation_long_time_series(f,dudt,nperiod)[:,0,0]
fu = calc_correlation_long_time_series(f,u,nperiod)[:,0,0]
rr = calc_correlation_long_time_series(r,r,nperiod)[:,0,0]
rdudt = calc_correlation_long_time_series(r,dudt,nperiod)[:,0,0]
ru = calc_correlation_long_time_series(r,u,nperiod)[:,0,0]
t_cor = delta_t * np.arange(memoryf.shape[0])

***(iv-a) Check results: plot $\Omega$ and $\Gamma(t)$***  

In [ ]:
print("Markov coefficient Omega = ", omega)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Correlation time t")
ax.set_ylabel("Memory function $\Gamma(t)$")
ax.plot(t_cor,memoryf.real,label="Re[$\Gamma(t)$]")
ax.plot(t_cor,memoryf.imag,label="Im[$\Gamma(t)$]")
plt.legend()
plt.show()

***(iv-b) Check results: plot time evolution of $f(t)=\Omega u(t)+s(t)+r(t)$***  

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Time t")
ax.set_ylabel("$f(t)=\Omega u(t)+s(t)+r(t)$")
tsta=400
tend=450
ax.plot(t,f.real,label="Re[f(t)]")
ax.plot(t,(omega*u).real,label="Re[$\Omega$u(t)]")
ax.plot(t,s.real,label="Re[s(t)]")
ax.plot(t,r.real,label="Re[r(t)]")
ax.set_xlim(tsta,tend)
plt.legend()
plt.show()

***(iv-c) Check results: Extended generalized fluctuation-dissipation theorem $\Gamma(t) = \langle r(t) du/dt \rangle / \langle u u \rangle$***  

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Correlation time t")
ax.set_ylabel(r"$\Gamma(t) = \langle r(t) du/dt \rangle / \langle u u \rangle$")
ax.plot(t_cor,memoryf.real,label="Re[$\Gamma(t)$]")
ax.plot(t_cor,memoryf.imag,label="Im[$\Gamma(t)$]")
ax.plot(t_cor,(rdudt/uu[0]).real,label=r"Re[$\langle r(t) du/dt \rangle / \langle u u \rangle$]",dashes=[4,4],linewidth=4)
ax.plot(t_cor,(rdudt/uu[0]).imag,label=r"Im[$\langle r(t) du/dt \rangle / \langle u u \rangle$]",dashes=[4,4],linewidth=4)
plt.legend()
plt.show()

***(iv-d) Check results: $\langle r(t) u \rangle = 0$***  
Namely, the uncorrelated term $r(t)$ indeed extracts the part of $f(t)$ uncorrelated with $u$. It is checked by $\langle r(t) u \rangle \ll \langle f(t) u \rangle$.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Correlation time t")
ax.set_ylabel("Correlation$")
ax.plot(t_cor,ru.real,label=r"Re[$\langle r(t) u \rangle$]")
ax.plot(t_cor,ru.imag,label=r"Im[$\langle r(t) u \rangle$]")
ax.plot(t_cor,fu.real,label=r"Re[$\langle f(t) u \rangle$]")
ax.plot(t_cor,fu.imag,label=r"Im[$\langle f(t) u \rangle$]")
plt.legend()
plt.show()

***(v-a) Fitting the memory function $\Gamma(t)$***
$$
\Gamma(t) = \frac{\gamma}{\tau} \exp\left(-\frac{t}{\tau}\right)
$$
where $\gamma, \tau \in \mathbb{C}$.

In [ ]:
from mzprojection import memoryf_get_fitting_coef, memoryf_fitted
tau, cl = memoryf_get_fitting_coef(delta_t,memoryf,order=0,t_range=int(len(memoryf)/2))
gamma = cl[0]*tau
print("gamma=",gamma)
print("tau=",tau)

t_cor = delta_t*np.arange(len(memoryf))

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Correlation time t")
ax.set_ylabel(r"$\Gamma(t) = \langle r(t) du/dt \rangle / \langle u u \rangle$")
ax.axhline(0,color="k",ls="dashed")
ax.plot(t_cor,memoryf.real,label=r"Re[$\Gamma(t)$]")
ax.plot(t_cor,memoryf.imag,label=r"Im[$\Gamma(t)$]")
ax.plot(t_cor,memoryf_fitted(tau,cl,t_cor).real,label=r"Re[$\gamma/\tau*\exp(-t/\tau)$]")
ax.plot(t_cor,memoryf_fitted(tau,cl,t_cor).imag,label=r"Im[$\gamma/\tau*\exp(-t/\tau)$]")
# ax.plot(t_cor,(gamma/tau*np.exp(-t_cor/tau)).real,label=r"Re[$\gamma/\tau*\exp(-t/\tau)$]")
# ax.plot(t_cor,(gamma/tau*np.exp(-t_cor/tau)).imag,label=r"Im[$\gamma/\tau*\exp(-t/\tau)$]")
plt.legend()
plt.show()

***(v-b) Fitting the uncorrelated term $\langle r(t) r \rangle$***
$$
\langle r(t) r \rangle = \mathrm{Re} \left[\frac{\sigma^2}{\theta}\right] \exp\left(-\frac{t}{\theta}\right)
$$
where $\sigma \in \mathbb{R}$ and $\theta \in \mathbb{C}$.

In [ ]:
from mzprojection import rr_get_fitting_coef, rr_fitted
theta, sigma = rr_get_fitting_coef(delta_t,rr,t_range=int(len(rr)/2))
print("sigma=",sigma)
print("theta=",theta)

t_cor = delta_t*np.arange(len(rr))

fig = plt.figure()
ax = fig.add_subplot(111)
ax.set_xlabel("Correlation time t")
ax.set_ylabel(r"$\Gamma(t) = \langle r(t) du/dt \rangle / \langle u u \rangle$")
ax.axhline(0,color="k",ls="dashed")
ax.plot(t_cor,rr.real,label=r"Re[$<r(t)r^*>$]")
ax.plot(t_cor,rr.imag,label=r"Im[$<r(t)r^*>$]")
ax.plot(t_cor,rr_fitted(theta,sigma,t_cor).real,label=r"Re[$\sigma^2/\theta*\exp(-t/\theta)$]")
ax.plot(t_cor,rr_fitted(theta,sigma,t_cor).imag,label=r"Im[$\sigma^2/\theta*\exp(-t/\theta)$]")
# ax.plot(t_cor,((sigma**2/theta).real*np.exp(-t_cor/theta)).real,label=r"Re[$\sigma^2/\theta*\exp(-t/\theta)$]")
# ax.plot(t_cor,((sigma**2/theta).real*np.exp(-t_cor/theta)).imag,label=r"Im[$\sigma^2/\theta*\exp(-t/\theta)$]")
plt.legend()
plt.show()

In [ ]:
print("Omega=",omega)
print("gamma=",gamma)
print("  tau=",tau)
print("sigma=",sigma)
print("theta=",theta)